<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_rec_sys/blob/main/coursework/cw_rec_sys_glvv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Курсовик по предмету "Рекомендательные системы" Гладышев В.В.

### EDA, фильтрация, feature engineering:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/coursework/cw_eda_glvv.ipynb

### библиотека с функциями метрик:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/metrics.py

### подготовка данных: 

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/utils.py

### класс с моделями:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/recommenders.py

In [1]:
pip install implicit

     |████████████████████████████████| 1.1MB 8.4MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419492 sha256=ad0e08a37e12334e7104a6f739c11f9ca38dd3afcfb3cbc00924d78ba2445233
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [2]:
pip install lightfm

     |████████████████████████████████| 317kB 8.0MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp36-cp36m-linux_x86_64.whl size=704268 sha256=3ff13bdabdaed741e2d346e1ae6723dbf4b2b12f0b52d1bdc56ac5ec595f9925
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [39]:
path_to_src = '/content/gdrive/My Drive/Colab Notebooks/GB Rec Sys/src/'

In [40]:
if path_to_src not in sys.path:
    sys.path.append(path_to_src)

# Написанные функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

In [41]:
path_to_files = '/content/gdrive/My Drive/Colab Notebooks/GB Rec Sys/data'

In [42]:
data          = pd.read_csv(os.path.join(path_to_files, "data.csv"))
user_features = pd.read_csv(os.path.join(path_to_files, "user_features.csv"))
item_features = pd.read_csv(os.path.join(path_to_files, "item_features.csv"))

In [43]:
# схема обучения и валидации
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [44]:
recommender = MainRecommender(data_train_lvl_1, item_features, user_features)

user_item_matrix
user_feat
item_feat


ALS model


ALS TF-IDF model


ALS BM25 model
LightFM model
Own


In [63]:
users_lvl_2_pred_als = pd.DataFrame(data_val_lvl_2['user_id'].unique())
users_lvl_2_pred_als.columns = ['user_id']

In [64]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

In [65]:
k_n = 5
num = 200

In [66]:
popularity = data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
item_list = popularity.sort_values('n_sold', ascending=False).head(num+1).item_id.tolist()
item_list = item_list[1:num+1]
len(item_list)

200

In [67]:
user_train_list = data_train_lvl_1["user_id"].unique().tolist()

In [68]:
def get_pred_als(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [69]:
def get_pred_als_tfidf(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_tfidf_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [70]:
def get_pred_als_bm25(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_bm25_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [71]:
def get_pred_own(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_own_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [72]:
def get_pred_similar_items(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_similar_items_recommendation(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [73]:
users_lvl_2_pred_als['predictions'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec"].mean()

0.1831537708129261

In [74]:
users_lvl_2_pred_als["rec"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec"].mean()

0.03038223074316197

In [75]:
users_lvl_2_pred_als['predictions_tfidf'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als_tfidf(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec_tfidf"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_tfidf"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec_tfidf"].mean()

0.14466209598432772

In [76]:
users_lvl_2_pred_als["rec_tfidf"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_tfidf"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec_tfidf"].mean()

0.023503027007213566

In [77]:
users_lvl_2_pred_als['predictions_bm25'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als_bm25(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec_bm25"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_bm25"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec_bm25"].mean()

0.22047012732614774

In [78]:
users_lvl_2_pred_als["rec_bm25"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_bm25"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec_bm25"].mean()

0.03110276563513014

In [79]:
users_lvl_2_pred_als.head()

,user_id,predictions,prec,rec,predictions_tfidf,prec_tfidf,rec_tfidf,predictions_bm25,prec_bm25,rec_bm25
0,338,"[1026118, 995785, 1037863, 845208, 1133018, 11...",0.8,0.042105,"[845208, 1037863, 1026118, 995785, 1041259, 11...",0.8,0.042105,"[1026118, 845208, 1037863, 1082185, 995785, 83...",0.8,0.042105
1,2120,"[1082185, 1029743, 995242, 981760, 1106523, 11...",0.0,0.000000,"[1082185, 1029743, 995242, 981760, 1106523, 90...",0.0,0.000000,"[1870702, 201704, 1313189, 43871, 1082185, 849...",0.0,0.000000
2,2324,"[1082185, 5978656, 981760, 995242, 1029743, 99...",0.6,0.060000,"[1082185, 1053690, 1118393, 981760, 1080414, 9...",0.2,0.020000,"[1082185, 1127831, 961554, 860776, 981760, 986...",0.4,0.040000
3,514,"[5590287, 5588666, 5584027, 5585635, 878996, 5...",0.6,0.061224,"[5584027, 5590287, 5588666, 5585639, 5585727, ...",0.2,0.020408,"[1082185, 5585735, 878996, 5584808, 5590287, 5...",0.6,0.061224
4,1762,"[886703, 1136257, 1108168, 918335, 916381, 108...",0.2,0.010000,"[874736, 893018, 866172, 5591170, 968687, 1071...",0.2,0.010000,"[826249, 1098066, 883404, 1029743, 1071939, 85...",0.4,0.020000


### Строим классификатор

In [88]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: get_pred_als_bm25(recommender, user_train_list, item_list, x, num=num))

In [89]:
users_lvl_2.head()

,user_id,candidates
0,2070,"[1082185, 1080414, 1085604, 1055863, 839147, 8..."
1,2021,"[981760, 951590, 1133018, 845208, 1098066, 896..."
2,1753,"[1080414, 879755, 1085604, 1051069, 1055863, 9..."
3,2120,"[1870702, 201704, 1313189, 43871, 1082185, 849..."
4,1346,"[1082185, 1870702, 1127831, 1313189, 201704, 8..."


In [91]:
users_lvl_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154 entries, 0 to 2153
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     2154 non-null   int64 
 1   candidates  2154 non-null   object
dtypes: int64(1), object(1)
memory usage: 33.8+ KB


In [92]:
df=pd.DataFrame({'user_id':users_lvl_2.user_id.values.repeat(len(users_lvl_2.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2.candidates.values)})

In [95]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

In [96]:
targets_lvl_2.shape

(445214, 3)

In [97]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left') #
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left') #

targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,bill_avg,week_last_bill,items_per_week
0,2070,1082185,1.0,2,PRODUCE,National,TROPICAL FRUIT,BANANAS,40 LB,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.891039,95.0,29.814815
1,2070,1080414,1.0,693,DRUG GM,National,CANDY - CHECKLANE,CANDY BARS (SINGLES)(INCLUDING,1.6 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.891039,95.0,29.814815


In [98]:
targets_lvl_2.shape

(445214, 19)

In [99]:
targets_lvl_2.fillna(0, inplace=True)

In [100]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [101]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'bill_avg',
 'week_last_bill',
 'items_per_week']

In [110]:
lgb = LGBMClassifier(objective='binary', 
                     n_estimators=1500,
                     learning_rate=0.008,
                     max_depth=7, 
                     verbose=1,
                     categorical_column=cat_feats)

hist = lgb.fit(X_train, y_train, 
               eval_set=[(X_train, y_train)],
               eval_metric=['logloss'],
               early_stopping_rounds=50,
               verbose=0)

train_preds = lgb.predict_proba(X_train)

In [111]:
hist.evals_result_['training']['binary_logloss'][-10:]

[0.2394399530169738,
 0.23941679094679655,
 0.2394071469635213,
 0.2393943837136871,
 0.2393878287549422,
 0.23937648547668872,
 0.23936530068709705,
 0.239353367046835,
 0.2393454724012716,
 0.23933981410888594]

In [112]:
classifer_prediction = X_train[['user_id', 'item_id']]
classifer_prediction["pred"] = train_preds[:, 1]
classifer_prediction.tail()

,user_id,item_id,pred
445209,1745,1069348,0.006226
445210,1745,874563,0.003222
445211,1745,1008400,0.003324
445212,1745,1115335,0.006348
445213,1745,1124475,0.010186
